# Análise do GRASP MCMC

Para rodar é necessário estar em uma máquina em que o GRASP roda rápido (poucos segundos). Deve-se customizar o caminho do executável em `./src/grasp.py` e, no caso do GRASP profissional, é necessário utilizar um arquivo TCI compatível (`../grasp/STANDARD/TICRA_TOOLS/BINGO_TICRA_TOOLS.tci`) na chamada da função `run_grasp`.

## Inicialização

In [1]:
import os
import sys
import glob
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
#-----
import time
import subprocess
import emcee
from getdist import plots as gdplt
from getdist import MCSamples
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.path.split(os.getcwd())[0], "src")))
import grasp
#-----
%matplotlib inline
plt.rcParams["figure.figsize"] = (16, 6)
InteractiveShell.ast_node_interactivity = "all"

## Emcee Analysis

In [2]:
# Gera arquivo tor para translação em x do feed e do seundário.
def make_tor_mcmc(*thetas):
    dirname = "../grasp/STANDARD/tmp_mcmc/"
    # cria pasta
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    filename = dirname + str(np.random.randint(10000000)) + ".tor"
    feed_string1 = grasp.move_feed(x=thetas[0])
    # grava arquivo tor
    grasp._make_tor(filename, feed_string1, "../grasp/STANDARD/BINGO_CUT_fast.tor", 15, 7)
    sec_string = grasp.translate_secondary(x=thetas[1])
    grasp._make_tor(filename, sec_string, filename, 7, 0)
    return filename

In [3]:
# COnfiguração fiducial
df00 = grasp.read_cut("../grasp/STANDARD/tmp_mcmc/cut00.cut")
best_params = [0, 0]

def residual(df):
    FWHM = grasp.get_FWHM(df)
    gain_max = grasp.gain_max(df, FWHM)
    FWHM_0 = grasp.get_FWHM(df00)
    gain_0 = grasp.gain_max(df00, FWHM)
    result = - np.abs(gain_max - gain_0)/gain_0
    return result
    
    
def log_prior(thetas):
    if (thetas[0] < 0.1) and (thetas[0] > -0.1):
        result = 0
    else:
        result = -np.inf
    return result


def log_likelyhood(thetas):
    tor_file = make_tor_mcmc(*thetas)
    _ = grasp.run_grasp(tor_file, 
                    gpxfile="../grasp/STANDARD/batch.gxp", 
                    tcifile="../grasp/STANDARD/BINGO_SIMPLES.tci", 
                    daemon=False)
    cut_file = ".." + tor_file.split(".")[-2] + ".cut"
    df = grasp.read_cut(cut_file)
    result = - residual(df)
    return result


def log_posterior(thetas):
    result = log_prior(thetas) + log_likelyhood(thetas)
    return result

In [4]:
ndim = len(best_params)
nburn = 1
nwalkers = 3
nsteps = 20
starting_guesses = np.random.normal(loc=0, scale=0.1, size=(nwalkers, ndim) )

In [6]:
filename = "../grasp/STANDARD/tmp_mcmc/grasp_chains.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, backend=backend)
sampler.run_mcmc(starting_guesses, nsteps, progress=True);

emcee: Exception while calling your likelihood function:
  params: [-0.39149827 -0.08841298]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/opt/miniconda3/envs/pibic/lib/python3.9/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/tmp/ipykernel_308041/3783869664.py", line 34, in log_posterior
    result = log_prior(thetas) + log_likelyhood(thetas)
  File "/tmp/ipykernel_308041/3783869664.py", line 23, in log_likelyhood
    _ = grasp.run_grasp(tor_file,
  File "/home/lbarosi/BAROSI/HOME/6000_PYTHONIA/2_Doing/EMSimulationGrasp/src/grasp.py", line 279, in run_grasp
    result = run_command(command)
  File "/home/lbarosi/BAROSI/HOME/6000_PYTHONIA/2_Doing/EMSimulationGrasp/src/grasp.py", line 381, in run_command
    out, err = process.communicate()
  File "/opt/miniconda3/envs/pibic/lib/python3.9/subprocess.py", line 1134, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/opt/miniconda3/envs/pibic/lib/python3.9/subprocess.py", line 1979, in _communicate
    ready = selector.select(timeo

KeyboardInterrupt: 

In [ ]:
reader = emcee.backends.HDFBackend(filename)
samples = reader.get_chain(flat=True)
samples = MCSamples(samples=samples,names = ["a", "b"], labels = ["a", "b"])

g = gdplt.get_subplot_plotter()
g.settings.figure_legend_frame = False
g.settings.alpha_filled_add=0.4
g.settings.title_limit_fontsize = 14
g.triangle_plot(samples, ['a', 'b'], 
    filled=True, 
    legend_loc='upper right', 
    title_limit=3
               )
plt.show();